# Boss 直聘自动化 - 完整功能演示

本 Notebook 提供 Boss 直聘自动化系统的完整功能演示，包括：

## 🚀 **核心功能**
- **智能简历处理**：多种捕获方法（WASM、Canvas拦截、截图）
- **AI决策系统**：结合岗位要求与简历进行智能匹配
- **批量处理**：支持多候选人并发处理
- **OCR集成**：本地和云端OCR处理
- **通知系统**：DingTalk集成

## 📋 **API功能演示**
- 服务状态检查与健康监控
- 消息列表获取与聊天历史
- 简历请求与在线简历查看
- 多种简历捕获方法演示
- 图像处理与OCR转换
- 批量操作与并发处理

## 🔧 **前置条件**
- Boss服务已运行 (`python start_service.py`)
- 可选环境变量：
  - `OPENAI_API_KEY`：AI决策功能
  - `DINGTALK_WEBHOOK`：通知功能
  - `DEMO_CHAT_ID`：演示用聊天ID


In [1]:
# 📦 导入依赖和初始化客户端
import os
import json
import time
from datetime import datetime
from typing import Dict, List, Optional
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64

from boss_client import BossClient, ResumeResult

# 配置
BASE_URL = os.environ.get('BOSS_SERVICE_URL', 'http://127.0.0.1:5001')
DEMO_CHAT_ID = os.environ.get('DEMO_CHAT_ID', '46232784-0')  # 演示用聊天ID

print("🚀 Boss 直聘自动化系统 - 完整功能演示")
print("=" * 60)

# 初始化客户端
client = BossClient(BASE_URL)

# 检查服务状态
print("🔍 检查服务状态...")
status = client.get_status()
print(f"✅ 服务状态: {status.get('status', 'unknown')}")
print(f"📊 浏览器状态: {status.get('browser_status', 'unknown')}")
print(f"⏰ 启动时间: {status.get('startup_time', 'unknown')}")

# 健康检查
if client.is_service_healthy():
    print("✅ 服务健康检查通过")
else:
    print("❌ 服务健康检查失败")
    raise RuntimeError("服务不可用，请先启动服务 (python start_service.py)")

print("\n🔄 服务重启功能演示...")
# 注意：实际环境中谨慎使用重启功能
# restart_result = client.restart()
# print(f"重启结果: {restart_result}")
DEMO_CHAT_ID

🚀 Boss 直聘自动化系统 - 完整功能演示
🔍 检查服务状态...
✅ 服务状态: running
📊 浏览器状态: unknown
⏰ 启动时间: unknown
✅ 服务健康检查通过

🔄 服务重启功能演示...


'46232784-0'

## 🔧 **基础服务操作**

演示基本的服务管理功能


In [2]:
# 🔧 基础服务操作演示

print("📋 获取通知列表...")
notifications = client.get_notifications(limit=5)
print(f"📊 通知数量: {len(notifications.get('notifications', []))}")
if notifications.get('notifications'):
    print("📝 最新通知:")
    for notif in notifications['notifications'][-3:]:
        print(f"   - {notif.get('message', 'N/A')} ({notif.get('level', 'info')})")

print("✅ 基础服务操作演示完成")


📋 获取通知列表...
📊 通知数量: 5
📝 最新通知:
   - 事件管理器设置成功 (success)
   - 已导航到聊天页面 (info)
   - 持久化浏览器会话启动成功！ (success)
✅ 基础服务操作演示完成


## 💬 **消息管理功能**

演示消息列表获取、聊天历史查看等功能


In [3]:
# 💬 消息管理功能演示

print("📋 获取消息列表...")
messages_response = client.get_messages(limit=10)
print(f"✅ 消息获取成功: {messages_response.get('success')}")
print(f"📊 消息数量: {len(messages_response.get('messages', []))}")

if messages_response.get('success') and messages_response.get('messages'):
    messages = messages_response['messages']
    print(f"\n📝 消息列表 (前5条):")
    for i, msg in enumerate(messages[:5]):
        print(f"   {i+1}. {msg.get('candidate', 'Unknown')} - {msg.get('message', 'N/A')[:50]}...")
        print(f"      Chat ID: {msg.get('chat_id')} | 职位: {msg.get('job_title', 'N/A')}")
    
    # 使用第一个消息进行后续演示
    import random
    if messages:
        demo_chat_id = random.choice(messages)['chat_id']
        demo_candidate = messages[0].get('candidate', 'Unknown')
        print(f"\n🎯 选择演示候选人: {demo_candidate} (Chat ID: {demo_chat_id})")
        
        # 获取聊天历史
        print(f"\n📜 获取聊天历史...")
        history_response = client.get_chat_history(demo_chat_id)
        if history_response.get('success'):
            history = history_response.get('messages', [])
            print(f"📊 历史消息数量: {len(history)}")
            if history:
                print(f"📝 最新消息: {history[-1].get('content', 'N/A')[:100]}...")
        else:
            print(f"❌ 获取聊天历史失败: {history_response.get('error')}")
else:
    print("❌ 没有找到消息，请确保有活跃的聊天")
    demo_chat_id = DEMO_CHAT_ID
    print(f"🎯 使用默认Chat ID: {demo_chat_id}")

print("✅ 消息管理功能演示完成")

📋 获取消息列表...
✅ 消息获取成功: True
📊 消息数量: 10

📝 消息列表 (前5条):
   1. 张诚 - 张诚-大模型NLP-10年以上.pdf...
      Chat ID: 687609714-0 | 职位: 大模型算法工程师
   2. 曾勇平 - 您好，可以聊聊吗？您这个职位我很有兴趣，希望进一步了解...
      Chat ID: 699254525-0 | 职位: 大模型算法工程师
   3. 白高峰 - BOSS您好，我于天津大学的计算机及应用专业硕士毕业，拥有岗位相关的专业知识和技能，行业经验丰富。可...
      Chat ID: 720958213-0 | 职位: 大模型算法工程师
   4. 黄耀辉 - 您好，简历已发...
      Chat ID: 642482457-0 | 职位: 大模型算法工程师
   5. 康鸿 - 您好，本人有一段大模型算法工作经历，两段大模型算法的项目经历，熟悉大模型的微调以及RAG和agent...
      Chat ID: 120300315-0 | 职位: 大模型算法工程师

🎯 选择演示候选人: 张诚 (Chat ID: 720958213-0)

📜 获取聊天历史...
📊 历史消息数量: 0
✅ 消息管理功能演示完成


## 💬 **发送消息功能**

演示发送文本消息到指定对话


In [4]:
# 发送消息演示
if demo_chat_id:
    print(f"📤 发送消息到 Chat ID: {demo_chat_id}")
    
    # 发送测试消息
    test_message = "您好，我对您的简历很感兴趣，希望能进一步沟通。"
    send_result = client.send_message(demo_chat_id, test_message)
    
    if send_result.get('success'):
        print(f"✅ 消息发送成功: {send_result.get('details')}")
        print(f"📝 发送内容: {test_message}")
    else:
        print(f"❌ 消息发送失败: {send_result.get('error')}")
        
    # 等待一下再获取聊天历史验证
    time.sleep(2)
    print("\n📜 验证消息是否发送成功...")
    history_response = client.get_chat_history(demo_chat_id)
    if history_response.get('success'):
        history = history_response.get('messages', [])
        print(f"📊 更新后历史消息数量: {len(history)}")
        if history:
            latest_message = history[-1]
            print(f"📝 最新消息: {latest_message.get('content', 'N/A')[:100]}...")
    else:
        print(f"❌ 获取聊天历史失败: {history_response.get('error')}")
else:
    print("❌ 没有可用的Chat ID进行消息发送演示")
    
print("✅ 发送消息功能演示完成")


📤 发送消息到 Chat ID: 720958213-0
✅ 消息发送成功: 消息发送成功
📝 发送内容: 您好，我对您的简历很感兴趣，希望能进一步沟通。

📜 验证消息是否发送成功...
📊 更新后历史消息数量: 0
✅ 发送消息功能演示完成


In [30]:
# demo_chat_id = '687609714-0'
print(f"\n🔍 在线简历演示({demo_chat_id})")

print("\n2️⃣ WASM模式捕获...")
result_wasm = client.view_online_resume(demo_chat_id)
print(f"✅ WASM模式结果: {result_wasm.success}")
if not result_wasm.success:
    print(f"❌ WASM模式失败: {result_wasm.details}")


print("✅ 简历处理功能演示完成")
if isinstance(result_wasm.text, str):
    print(result_wasm.text[:200])
else:
    print(result_wasm.text)



🔍 在线简历演示(720958213-0)

2️⃣ WASM模式捕获...
✅ WASM模式结果: True
✅ 简历处理功能演示完成
白高峰
 刚刚活跃
47岁  10年以上  硕士  在职-月内到岗
• 工作认真、负责，具有良好的合作精神
• 为人正直，执行力强
• 有创新精神，应用型创新系列成果丰硕

期望职位
---
北京Python行业不限面议

工作经历
---
出门问问信息科技有限公司
C/C++
2017.01 - 2025.08
先后工作于车载语音SDK和AI 算法中台团队，分别从事：
· 语音SDK架构及相关功


In [10]:
print("📋 简历请求功能演示...")
# 请求简历
request_result = client.request_resume(demo_chat_id)
print(f"✅ 简历请求结果: {request_result.get('success')}")
if request_result.get('success'):
    print(f"📝 请求消息: {request_result.get('details', 'N/A')}")
else:
    print(f"❌ 请求失败: {request_result.get('error')}")



📋 简历请求功能演示...
✅ 简历请求结果: True
📝 请求消息: 简历请求已发送（按钮已禁用）


In [12]:
# 查看离线简历演示
chat_id = '643094543-0'
print(f"📄 查看简历 Chat ID: {chat_id}")
    
# 点击查看附件简历
view_result = client.view_resume(chat_id)

if view_result.get('success'):
    print(f"✅ 简历查看器已打开: {view_result.get('details')}")
    print(view_result['content'][:200])
else:
    print(f"❌ 查看简历失败: {view_result.get('error')}")
    print(f"📝 错误详情: {view_result}")
    


📄 查看简历 Chat ID: 643094543-0
✅ 简历查看器已打开: 简历查看器已打开
个人简历

基本信息

姓名： 金久五出生年月： 1991-01

性别： 男电话： 198-6372-0995

邮箱： jwjin200@163.com微信：JJW910106

我是一名统计学研究生，在机器学习、统计建模和优化算法拥有丰富和深厚的研究经验。在研究生学习期间，我在顶级期刊

Annals of Statistics上发表了一篇论文。通过在美国顶级银行Wells Fargo的实习


## 📄 **简历请求功能**

演示简历请求、多种捕获方法、OCR处理等功能


## 🤖 **AI决策系统**

演示结合岗位要求和简历进行智能匹配决策


In [14]:
# 读取岗位要求 YAML
import yaml, json
with open('jobs/criteria.yaml','r',encoding='utf-8') as f:
    criteria = yaml.safe_load(f)
criteria


{'roles': [{'id': 'default',
   'position': '大模型算法工程师',
   'description': '负责大语言模型/多模态模型的训练、微调与落地；\n参与推理性能优化与工程化部署；\n与业务协作将模型能力嵌入产品。\n',
   'target_profile': '- 计算机/数学/电子工程等相关专业，硕士及以上优先；\n- 熟悉 Transformer/Lora/PEFT 等；\n- 3-8 年算法或相关经验，具备端到端项目经验。\n',
   'filters': {'must_have': ['Python',
     {'深度学习框架': ['PyTorch', 'TensorFlow']},
     'LLM/LMM 相关经验'],
    'nice_to_have': ['C++', '分布式训练(DeepSpeed/Megatron)', '向量数据库/检索增强(RAG)'],
    'must_not': ['仅运维/测试背景', '与岗位无关的实习仅有']},
   'keywords': {'positive': ['大模型',
     'LLM',
     'Transformer',
     'Fine-tuning',
     'LoRA',
     'RLHF',
     '知识蒸馏',
     'RAG'],
    'negative': ['爬虫', '外包', '纯NLP规则']},
   'scoring': {'weights': {'experience': 0.35,
     'skills_match': 0.35,
     'projects': 0.2,
     'education': 0.1},
    'threshold': {'greet': 0.7, 'borderline': 0.6}},
   'notes': '可根据岗位变体微调 filters/keywords/weights。\n'}]}

## 推荐牛人列表动作

In [1]:
# 👥 查看推荐候选人
print("🔍 获取推荐候选人列表...")

# 调用推荐候选人接口
result = client.get_recommended_candidates(limit=5)

if result.get('success'):
    candidates = result.get('candidates', [])
    print(f"✅ 成功获取 {len(candidates)} 个推荐候选人")
    print()
    
    # 显示候选人信息
    for i, candidate in enumerate(candidates, 1):
        print('-' * 20 + f' 第{i}个候选人 ' + '-' * 20)
        print(candidate)
else:
    print(f"❌ 获取推荐候选人失败: {result}")
    

🔍 获取推荐候选人列表...


NameError: name 'client' is not defined

In [29]:
# 查看候选人简历
result = client.view_recommended_candidate_resume(2)
print(result)

{'success': False, 'error': 'Request timeout'}


## 🗑️ **丢弃候选人功能**

演示丢弃候选人的功能 - 点击"不合适"按钮


In [ ]:
# 丢弃候选人演示
demo_chat_id = '643094543-0'  # 使用一个测试用的Chat ID
print(f"🗑️ 丢弃候选人 Chat ID: {demo_chat_id}")

# 丢弃候选人
discard_result = client.discard_candidate(demo_chat_id)

if discard_result.get('success'):
    print(f"✅ 候选人已丢弃: {discard_result.get('details')}")
    print(f"📝 操作时间: {discard_result.get('timestamp')}")
else:
    print(f"❌ 丢弃候选人失败: {discard_result.get('error')}")
    print(f"📝 错误详情: {discard_result.get('details')}")

print("✅ 丢弃候选人功能演示完成")
